In [1]:
#from gpt4all import GPT4All
#model = GPT4All("orca-mini-3b-gguf2-q4_0.gguf")
#output = model.generate("The capital of France is ", max_tokens=3)
#print(output)

In [2]:
!pip install llm
!pip install numba
!pip install gpt4all
!pip install sentence_transformers
!pip install pypdf2
!pip install pdfplumber
!pip install langchain
!pip install langchain.text_splitter


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement langchain.text_splitter (from versions: none)
ERROR: No matching distribution found for langchain.text_splitter


In [ ]:
from PyPDF2 import PdfReader
from pdfplumber import pdf
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer, util
import time
from gpt4all import GPT4All

import sys
import random

def progressBar(count_value, total, suffix=''):
    bar_length = 100
    filled_up_Length = int(round(bar_length * count_value / float(total)))
    percentage = round(100.0 * count_value / float(total), 1)
    bar = '=' * filled_up_Length + '-' * (bar_length - filled_up_Length)
    sys.stdout.write('[%s] %s%s ...%s\r' % (bar, percentage, '%', suffix))
    sys.stdout.flush()

def get_query():
    query = input("Enter your question (or type 'exit' to quit):\n")
    progressBar(1, 7)
    return query

def load_split_pdf(pdf_path):
    pdf_loader = PdfReader(open(pdf_path, "rb"))
    pdf_text = ""
    for page_num in range(len(pdf_loader.pages)):
        pdf_page = pdf_loader.pages[page_num]
        pdf_text += pdf_page.extract_text()
    progressBar(2, 7)
    return pdf_text

def split_text_using_RCTS(pdf_text):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=2048,
        chunk_overlap=64
    )
    split_texts = text_splitter.split_text(pdf_text)
    paragraphs = []
    for text in split_texts:
        paragraphs.extend(text.split('\n'))
    progressBar(3, 7)
    return paragraphs

def Initialize_sentence_transformer():
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
    embeddings = SentenceTransformer(model_name)
    progressBar(4, 7)
    return embeddings

def encode_each_paragraph(paragraphs, embeddings):
    responses = []
    for paragraph in paragraphs:
        response = embeddings.encode([paragraph], convert_to_tensor=True)
        responses.append((paragraph, response))
    progressBar(5, 7)
    return responses

def choose_most_relevant_sentence(embeddings, responses, query):
    query_embedding = embeddings.encode([query], convert_to_tensor=True)
    answers = []

    for paragraph, response in responses:
        similarity = util.pytorch_cos_sim(query_embedding, response).item()
        if similarity >= 0.6:
            answers.append(paragraph)
    answer = "\n".join(answers)
    progressBar(6, 7)
    return answer

def query_the_llm(answer, llm_model, query):
    prompt_message = answer + "\n" + query
    final_response = llm_model.generate(prompt=prompt_message)
    return final_response

def main(llm):
    pdf_path = "./mydoc.pdf"
    
    while True:
        query = get_query()
        
        if query.lower() == 'exit':
            print("Exiting...")
            break
        
        pdf_text = load_split_pdf(pdf_path)
        paragraphs = split_text_using_RCTS(pdf_text)
        #print (paragraphs)
        
        embeddings = Initialize_sentence_transformer()
        responses = encode_each_paragraph(paragraphs=paragraphs, embeddings=embeddings)
        
        answer = choose_most_relevant_sentence(embeddings=embeddings, responses=responses, query=query)
        final_response = query_the_llm(answer=answer, llm_model=llm, query=query)
        
        print("The answer from the model is:\n", final_response)
        
    print("Program finished.")
    progressBar(7, 7)

if __name__ == "__main__":
    llm = GPT4All("orca-mini-3b-gguf2-q4_0.gguf", allow_download=True)
    main(llm)


Enter your question (or type 'exit' to quit):
 what is the title


The answer from the model is:==========================================================--------------] 85.7% ...
  of the book that you are referring to?


Enter your question (or type 'exit' to quit):
 what is the book's title


The answer from the model is:==========================================================--------------] 85.7% ...
 ?
As an AI language model, I do not have access to your personal information or device. Please provide me with more details about the book you are referring to so that I can assist you better.


Enter your question (or type 'exit' to quit):
 where is nova studied?


...

The answer from the model is:==========================================================--------------] 85.7% ...
  in c++


Enter your question (or type 'exit' to quit):
 yes in c++


The answer from the model is:==========================================================--------------] 85.7% ...
 , i have a class called "MyClass" which has two member variables "x" and "y". I want to access the value of these members from within another class called "AnotherClass".
I can do this by creating an object of MyClass and then accessing the values of x and y like this:

int main() {
 AnotherClass anotherClass; // create an instance of AnotherClass
 int myValue = 10; // set a value for myValue
 
 // access x and y using object of MyClass
 auto myObject = myClass(); // create an object of MyClass
 std::cout << "x: " << myObject.getX() << ", y: " << myObject.getY() << std::endl; // print the values of x and y
 
 return 0;
}

However, I want to know if there is a way to access the value of the member


Enter your question (or type 'exit' to quit):
 give helloworld sample code in python


The answer from the model is:==========================================================--------------] 85.7% ...
 .

Here is a sample code that you can use to print "Hello World" on the console:

```python
print("Hello World")
```
